In [8]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
from IPython.display import display, HTML


In [9]:
#Setup for Google Drive and Google Sheets API
#Need to create a service account in Google Cloud to get the JSON key: https://console.cloud.google.com/iam-admin/serviceaccounts
#Need to enable Google Sheets and Google Drive APIs for the JSON key to work

scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('willamete-valley-plant-list-b71c56838f40.json', scope)
client = gspread.authorize(creds)


In [10]:
#Load data from Google Sheets

sheet = client.open("Willamette Valley Multifunctional Plant List").worksheet("Fruit Trees")
data = sheet.get_all_records()
df = pd.DataFrame(data)

In [11]:
#Set dataframe parameters

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 3)
pd.set_option('display.max_rows', None)  

In [12]:
# Set index for dataframe

df.set_index("Botanical name", inplace=True)
df.index.name = None 

In [13]:
#images for shade tolerance
ground_cover_shade_images = {
    'Full sun':'/images/full-shade.png',
    'Partial sun':'/images/full-sun.png',
    'Full shade':'/images/full-shade.png',
    'Full sun/Partial sun':'/images/full-sun-partial-sun.png',
    'Partial sun/Full shade':'/images/partial-sun-full-shade.png'
}

#images for columns that need checks
check_image = {
    'yes':'/images/check.png',
    'no' : ' ',
    'unknown' : 'Unknown'
}

#map images to Sun/Shade column
df['Sun/Shade'] = df['Sun/Shade'].map(ground_cover_shade_images)

# Convert image paths to HTML <img> tags
df['Sun/Shade'] = df['Sun/Shade'].apply(lambda x: f'<img src="{x}" width="30" height="30" />')


# Apply the function to each column that needs to be checked
for column in ['Native', 'Medicinal', 'Nitrogen fixer', 'Insect/Bee/Birds']:
    df[column] = df[column].map(check_image)
    df[column] = df[column].apply(lambda x: f'<img src="{x}" width="30" height="30" />' if x == '/images/check.png' else x)


In [14]:
#| label: plant-table

display(HTML(df.to_html(escape=False)))

,Common name,Class,Type,Height,Width,Sun/Shade,Soil Type,Propagation,Native,Water requirement,Maintenance,Bloom time,Fruit time,Medicinal,Nitrogen fixer,Insect/Bee/Birds,Other uses,Rate of growth,Flower color,Fall or winter cover,Pollinator habitat
Malus spp.,Apple,ST,D,5' - 50',4' - 30',,CL,G/S/R,,,R,,Summer/Fall,,,,,,,,
Pyrus communis,Pear,ST,D,7' - 50',6' - 25',,C/CL,G/S,,F,R,,Summer/Fall,,,,Invasive in wet prairie,,,,
Prunus spp.,Plum,ST,D,,,,CL,G/S/R,,,,,Summer,,,,,,,,
Pyrus spp.,Asian Pear,T,D,15' - 30',,,SL,G,,,R,,Summer/Fall,,,,,,,,
Prunus persica,Peach,ST,D,15' - 25',,,CL,G/S,,,R,,Summer,,,,,,L,,
Diospyrus virginiana,American persimmon,T,D,15' - 25',15' - 20',,C/CL,G/S,,,LN,,Fall,,,,Great dried,M,,,
Diospyrus kaki,Japanese persimmon,T,D,15' - 25',15' - 20',,C/CL,G,,,LNM,,Fall,,,,Great dried,M,,,
Prunus spp.,Cherry,,,,,,CL,G/S/R,,,M,,Summer,,,,,,,,
Ficus carica,Fig,ST,D,,,,CL/SL,G/L/C,,,M,,,,,,Excellent ice cream. Seeedlings of good stock a good bet,,,,
Asimina triloba,Pawpaw,T,D,10' - 40',10' - 20',,Many - WD best,Root cutting,,R,LN,Varies,Lt Summer/Erly Fall,,,,"Large tasty custard like fruit/leaves, bark, & seeds insecticidal (very) kills beneficialslast resort/fiber from inner bark/fairly productive in part shade/many cultivars/don't cook fruit",MF,H,PU,
